<a href="https://colab.research.google.com/github/AdamCorbinFAUPhD/CIRCLe-experiments/blob/main/resnext101/2023_01_20/CIRCLe_with_isic2018_with_skin_transformer_resnext101_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment notes

- date: 2023/01/16 8am
- base model: alexnet
- updating to calculate the normalization.


Results: 
-  didnt seem like it helped much


# Intro

This notebook is used to modify the implementation of CIRCLe from this paper : [CIRCLe: Color Invariant Representation
Learning for Unbiased Classification of Skin
Lesions](https://arxiv.org/pdf/2208.13528.pdf)

Their github repo is : https://github.com/arezou-pakzad/CIRCLe

This paper uses the Fitzpatrick17k dataset which can be obtained here: https://github.com/mattgroh/fitzpatrick17k

For these set of experiments we will use the ISIC 2017 dataset from: https://github.com/manideep2510/melanoma_segmentation.git 

#TODO list

1. [X] Download 2018 dataset
1. [X] Analize dataset to get Fitzpatrick info. 
1. [X] Save off Fitzpatrick info data so we dont have to do it every time
1. [X] load cached fitzpatrick data
1. [X] Create masks uing https://github.com/DebeshJha/2020-CBMS-DoubleU-Net Because Task 3 for 2018 doesnt havent masks. Trick was to get the higher end GPU and ram (12/29/2022)
1. [X] Create pytorch dataloader for ISIC 2018 dataset including loading masks, images, diagnossis, fitzpatrick type for training (12/30/2022) needed to create custom split function
1. [X] Create dataloaders for test and validation  (12/30/2022)
1. [X] Added jupiter notebook download code into the github repo (1/1/2023)
1. [X] plug in dataloader into CIRCLe main file (1/1/2023)
1. [X] Figure out how to transform image and mask the same from the dataloader (1/2/2023)
1. [X] Use the new dataloader to train the model (1/2/2023)
1. [X] Use new transformer for CIRCLe model (1/3/2023)
1. [ ] test using different base models
1. [ ] test that adding dropout might help with overfitting
1. [X] Add more metrics such as precision and recall (1/4/2023)
1. [ ] add fairness metrics
1. [ ] add confusion matrics
1. [ ] add sensitivity and specificity
1. [ ] add metrics for each class
1. [ ] (optional) Go back and download and use larger datasets
1. [ ] (optional) Run Fitzpatrick on larger datasets(currently using the test set from isic 2018 task 3)
1. [ ] The dataloaders need to be split stratified different than the current "training, validation, and test" as given from https://challenge.isic-archive.com/data/#2018 based on skin types. 12/30/2022 - I think this is done BUT we might consider doing k-fold approach which adds another layer of complexity to the dataloaders

# Set up the environment

In [1]:
!python --version

Python 3.8.10


## Installs & imports

## Download latest code

In [2]:
!git clone https://github.com/acorbin3/CIRCLe.git

Cloning into 'CIRCLe'...
remote: Enumerating objects: 491, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 491 (delta 83), reused 116 (delta 51), pack-reused 339
Receiving objects: 100% (491/491), 1.81 MiB | 22.34 MiB/s, done.
Resolving deltas: 100% (249/249), done.


In [3]:
%cd ./CIRCLe

/content/CIRCLe


In [4]:
!git checkout -- models/circle.py

In [5]:
!git pull

Already up to date.


In [6]:
!pip3 install -r ./requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 17.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Unins

**This next block of code will be needed if you get this error: **

A100-SXM4-40GB with CUDA capability sm_80 is not compatible with the current PyTorch installation. The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.

In [7]:
#!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

# IF ERROR, RESTART RUNTIME due to derm-ita lib
This is due to derm-ita using newer libaries than the Google Colab default(during this time of 12/24/2022)

# Train CIRCLe model 

In [8]:
%mkdir ./saved
%mkdir ./saved/model

In [28]:
!git checkout -- main.py
!git checkout -- datasets/isic_2018/dataset.py
!git checkout -- models/base.py
!git checkout -- Metrics.py

In [35]:
!git pull

Already up to date.


In [36]:
!python main.py --use_reg_loss False --base resnext101_32x8d --dataset isic2018 --hidden_dim 16

Flags:
	alpha: 0.1
	base: resnext101_32x8d
	batch_size: 32
	data_dir: ../data/fitz17k/images/all/
	dataset: isic2018
	epochs: 100
	gan_path: saved/stargan/
	hidden_dim: 16
	lr: 0.001
	model: circle
	model_save_dir: saved/model/
	num_classes: 7
	seed: 1
	use_reg_loss: True
	weight_decay: 0.001
isic2018 images already downloaded
isic 2018 masks already downladed
Donloading isic 2018 ground truth classification data
Creating dataframe
	 Looking for cached dataframe
		 datasets/isic_2018/saved_data_2022_12_27_isic_2018.csv
Creating dataframe. Complete!
Splitting up the dataset into train,test, validation datasets based on the skin condition
fizpatrick_skin_type: ('AKIEC', 1) 291
	 train 232
	 test 29
	 val 30
fizpatrick_skin_type: ('AKIEC', 2) 29
	 train 23
	 test 3
	 val 3
fizpatrick_skin_type: ('AKIEC', 3) 4
	 train 3
	 test 1
	 val 0
fizpatrick_skin_type: ('AKIEC', 4) 1
	 train 3
	 test 0
	 val 0
fizpatrick_skin_type: ('AKIEC', 6) 2
	 train 1
	 test 1
	 val 0
fizpatrick_skin_type: ('BCC

In [ ]:
#%mkdir /content/drive/MyDrive/Corbin_Adam_PhD_Workspace/corbin_papers/dissertation_proposal/model_checkpoints

In [ ]:
4

In [ ]:
#%cp ./saved/model/epoch97_acc_0.762.ckpt /content/drive/MyDrive/Corbin_Adam_PhD_Workspace/corbin_papers/dissertation_proposal/model_checkpoints/CIRCLE/mobilenetv3l/